In [2]:
!pip install scikit-learn
!pip install konlpy
!pip install numpy



In [120]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Hannanum
import numpy as np
import nltk
import pickle
# nltk.download('all')

hannanum = Hannanum()

def extract_keywords(text):
    # 텍스트에서 키워드 추출
    keywords = hannanum.nouns(text)
    eng_lower = text.lower()
    eng_keywords = nltk.word_tokenize(eng_lower)
    return " ".join(keywords + eng_keywords)  # 단어 리스트를 다시 문자열로 변환하여 반환

def calculate_similarity(a, b):
    # 직무와 직접적으로 관련 없는 stop words
    general_stopwords = ['경험', '능력', '경력', '기술', '업무', '작업', '능숙', '풍부', '향상', '다양', '다양한', '완료', '관련', '특화', '역량', '보유', '담당',
                         '성공', '성공적', '프로젝트', '분야', '활용', '스킬', '목표', '도전', '기록', '노력', '수행', '참여', '참가', '달성', '적용', '적응', '배움',
                         '기여', '협력', '활동', '향상', '성장', '발전']

    # 두 문서 벡터화
    vectorizer = TfidfVectorizer(stop_words=general_stopwords)
    tfidf_matrix = vectorizer.fit_transform([a, b])

    # 첫 번째 문서와 두 번째 문서의 TF-IDF 벡터 추출
    vector_a = tfidf_matrix[0]
    vector_b = tfidf_matrix[1]

    # 코사인 유사도 계산
    similarity = np.dot(vector_a, vector_b.T).toarray()[0, 0]

    # 유사도 보정
    similarity *= 3

    if similarity > 1:
      similarity = 1

    return similarity


def recommend_resumes(project_overview, resumes): # 이력서 추천
    # 업무 한 줄 소개로부터 키워드 추출
    project_keywords = extract_keywords(project_overview)

    # 이력서별로 유사도 계산
    similarity_scores = []
    for resume in resumes:
        # 이력서로부터 키워드 추출
        resume_keywords = extract_keywords(resume)
        # 업무 한 줄 소개와 이력서 사이의 유사도 계산
        similarity = calculate_similarity(project_keywords, resume_keywords)
        similarity_scores.append((resume, similarity))

    return similarity_scores

# 기업 사용자가 검색하는 업무 한 줄 소개
project_overview = "가상 현실 VR AR 게임 및 클라우드 플랫폼 경험을 가진 서버 개발자"
print("사용자 검색어: " + project_overview)

# ncs 게임 프로그래머 키워드
ncs = """게임프로그래머가 되기 위해서는 고등학교, 전문대학의 게임관련 정규 교육과정을 이수하거나 게임 인력을 전문적으로 양성하는 게임아카데미 등의 사설학원, 대학의 사회교육원에서 교육훈련을 받으면 된다.
대학에서는 게임관련학과가 있고, 특성화 고등학교에 컴퓨터게임제작과가 설치되어 있다.
이들 학과에서는 게임기획, 게임연출, 게임프로듀싱 및 아이디어 발굴을 위한 교육, 게임시나리오 작성, 아이디어 창출교육, 게임그래픽디자인, 게임프로그래밍, 게임음악 등을 교육한다.
게임 인력을 전문적으로 양성하는 사설교육기관에서는 3D애니메이션, 웹프로듀서, 디지털웹디자인, 게임프로그래머, 게임그래픽디자인 등을 실습 위주로 교육하고 있으며,
게임제작자가 되기 위해서는 기본적인 분야 중 하나 이상의 분야를 필수적으로 공부해야 한다.
Visual Tool, C언어 능숙자, 컴퓨터그래픽애니메이션 개발자나 실무경력 2∼3년의 경력자가 유리하며 윈도우, 포토샵, 프리미어, 프로그래밍언어(어셈블리, C/C++ 등), 자료구조 등에 대한 지식이 요구된다."""

사용자 검색어: 가상 현실 VR AR 게임 및 클라우드 플랫폼 경험을 가진 서버 개발자


In [122]:
# 업무 한 줄 소개와 ncs 결과 추출
search_result = recommend_resumes(project_overview, resumes)
#ncs_result = recommend_resumes(ncs, resumes)

percentage = (75, 25) # 검색어와 ncs의 반영비

print(project_overview)

# 반영비를 고려해 최종 점수 산출
final_scores = [0] * len(resumes)
for i in range(len(resumes)):
    final_scores[i] = (resumes[i], search_result[i][1])
    '''
for i in range(len(resumes)):
    final_scores[i] = (resumes[i], search_result[i][1] * percentage[0] + ncs_result[i][1] * percentage[1])'''

# 점수가 높은 순으로 정렬
final_resumes = sorted(final_scores, key=lambda x: x[1], reverse=True)

# 결과 출력
for resume, score in final_resumes:
    resume_num = resume.split('\n')[1]
    print(f"최종 점수: {score:.2f}, 이력서: {resume_num}")
    print(resume)

가상 현실 VR AR 게임 및 클라우드 플랫폼 경험을 가진 서버 개발자
최종 점수: 0.95, 이력서: 이력서 11

이력서 11

이름: 최승현
직군 및 직무: 게임 개발자
보유 스킬: Unreal Engine, C#, VR 게임 개발, AWS
경력사항: 
- JKL 엔터테인먼트 (3년)
  - 가상 현실(VR) 기반의 어드벤처 게임 '환상의 세계' 개발에 리드
  - VR 기술을 활용하여 현실과 가상 세계 간의 융합을 추구하여 긍정적인 사용자 리뷰 획득
  - AWS 클라우드 배포

최종 점수: 0.84, 이력서: 이력서 10

이력서 10

이름: 최승현
직군 및 직무: 게임 개발자
보유 스킬: Unreal Engine, C#, VR 게임 개발
경력사항: 
- JKL 엔터테인먼트 (3년)
  - 가상 현실(VR) 기반의 어드벤처 게임 '환상의 세계' 개발에 리드
  - VR 기술을 활용하여 현실과 가상 세계 간의 융합을 추구하여 긍정적인 사용자 리뷰 획득

최종 점수: 0.77, 이력서: 이력서 5

이력서 5

이름: 정호영
직군 및 직무: 게임 서버 엔지니어
보유 스킬: Java, Spring Boot, 네트워크 프로그래밍, 데이터베이스 설계, 클라우드
경력사항:

ServerCraft Solutions (2016년 ~ 2019년)
NetGaming Technologies (2019년 ~ 2022년)
TechPlay Studios (2022년 ~ 현재)
주요 성과:
서버 성능 최적화를 통한 게임 스무스한 플레이 환경 제공
실시간 멀티플레이어 기능 도입으로 사용자 상호작용 강화
주요 프로젝트:
"MMO Realm" (대규모 멀티플레이어 온라인 게임 서버)
"RealTime Battle Arena" (실시간 전투 멀티플레이어 게임)
"SocialHub Integration" (게임 내 소셜 기능 통합)
AWS 클라우드 플랫폼 개발

최종 점수: 0.68, 이력서: 이력서 12

이력서 12

이름: 손재민
직군 및 직무: 가상현실(VR) 서버 개발자
보유

In [107]:
# 시니어 사용자의 이력서 키워드
resumes = [
        """
이력서 1

이름: 김태영
직군 및 직무: 게임 프로그래머
보유 스킬: C++, Unity, Unreal Engine, 네트워킹, AI 프로그래밍
경력사항:

PixelCraft Studios (2016년 ~ 2019년)
TechFront Games (2019년 ~ 2022년)
FutureReality Entertainment (2022년 ~ 현재)
주요 성과:
Unity 및 Unreal Engine을 이용한 다양한 게임 모듈 구현
멀티플레이어 온라인 모드 도입으로 사용자 참여도 상승
주요 프로젝트:
"PixelQuest Adventures" (2D 플랫폼 어드벤처 게임)
"Galactic Tactics" (3D 우주 전략 게임)
"VR Exploration" (VR 탐험 어드벤처 게임)
""",
        """
이력서 2

이름: 이지수
직군 및 직무: 게임 디자이너
보유 스킬: Unity, C#, 3D 모델링, UI/UX 디자인
경력사항:

GameCraft Innovations (2018년 ~ 2021년)
DreamWorld Interactive (2021년 ~ 2023년)
Wonderland Games (2023년 ~ 현재)
주요 성과:
현실감 있는 VR 경험으로 사용자 평가 상승
게임 디자인과 UI/UX 개선으로 사용자 참여도 증가
주요 프로젝트:
"VRQuest" (VR 판타지 어드벤처 게임)
"Epic Conquerors Online" (멀티플레이어 온라인 전략 게임)
"Mystic Runes" (모바일 VR 어드벤처 게임)
""",
        """
이력서 3

이름: 백승호
직군 및 직무: 게임 아티스트
보유 스킬: Adobe Photoshop, Blender, 게임 아트 디자인, 캐릭터 디자인
경력사항:

ArtCraft Studios (2017년 ~ 2020년)
CreativeRealms Games (2020년 ~ 2022년)
ImaginationForge (2022년 ~ 현재)
주요 성과:
고품질 캐릭터 및 배경 아트 제작으로 게임 세계관 구축
다양한 게임 장르에 적합한 아트 스타일 개발
주요 프로젝트:
"FantasyWorld" (모바일 MMORPG)
"PixelPuzzle" (2D 퍼즐 게임)
"SpaceExplorers" (우주 탐험 게임)
""",
        """
이력서 4

이름: 송민지
직군 및 직무: 게임 테스터
보유 스킬: 게임 테스트, 버그 리포팅, 테스트 자동화
경력사항:

GameTesters Inc. (2019년 ~ 2021년)
BugHunters Co. (2021년 ~ 2023년)
QualityGaming Solutions (2023년 ~ 현재)
주요 성과:
품질 관리 프로세스 개선으로 게임 출시 전 품질 향상
테스트 자동화 도입으로 반복 작업 최소화
주요 프로젝트:
다양한 플랫폼에서의 게임 테스트 수행
게임 내 버그 및 이슈 식별 및 리포팅
""",
        """
이력서 5

이름: 정호영
직군 및 직무: 게임 서버 엔지니어
보유 스킬: Java, Spring Boot, 네트워크 프로그래밍, 데이터베이스 설계, 클라우드
경력사항:

ServerCraft Solutions (2016년 ~ 2019년)
NetGaming Technologies (2019년 ~ 2022년)
TechPlay Studios (2022년 ~ 현재)
주요 성과:
서버 성능 최적화를 통한 게임 스무스한 플레이 환경 제공
실시간 멀티플레이어 기능 도입으로 사용자 상호작용 강화
주요 프로젝트:
"MMO Realm" (대규모 멀티플레이어 온라인 게임 서버)
"RealTime Battle Arena" (실시간 전투 멀티플레이어 게임)
"SocialHub Integration" (게임 내 소셜 기능 통합)
AWS 클라우드 플랫폼 개발
""",
        """
이력서 6

이름: 홍길동
직군 및 직무: 게임 개발자
보유 스킬: C++, Unity, 3D 모델링
경력사항:
- ABC 게임스튜디오 (3년)
  - '모험의 시작' 게임에서 주요 캐릭터 및 맵 디자인 참여
  - 게임 내의 기능 개선과 새로운 기능 추가에 기여
""",
        """
이력서 7

이름: 김영희
직군 및 직무: 게임 서버 개발자
보유 스킬: Java, Spring Boot, MySQL
경력사항:
- XYZ 서버테크놀로지 (4년)
  - 대규모 다중 플레이어 온라인 게임의 서버 아키텍처 설계 및 구현
  - 서버 성능 향상을 위한 효율적인 데이터베이스 관리
""",
        """
이력서 8

이름: 박철수
직군 및 직무: 게임 개발자
보유 스킬: Python, Pygame, 게임 알고리즘
경력사항:
- DEF 게임랩 (2년)
  - 모바일 퍼즐 게임 '퍼즐마스터'의 핵심 로직 개발 및 업데이트
  - 게임 유저들과의 피드백을 수용하여 사용자 경험 향상
""",
        """
이력서 9

이름: 이지은
직군 및 직무: 게임 서버 개발자
보유 스킬: Node.js, MongoDB, 웹 소켓
경력사항:
- GHI 클라우드게임즈 (5년)
  - 실시간 멀티플레이어 게임의 서버 성능 최적화 및 확장
  - 안정적인 서버 운영을 위한 모니터링 및 유지보수 프로세스 개발
""",
        """
이력서 10

이름: 최승현
직군 및 직무: 게임 개발자
보유 스킬: Unreal Engine, C#, VR 게임 개발
경력사항:
- JKL 엔터테인먼트 (3년)
  - 가상 현실(VR) 기반의 어드벤처 게임 '환상의 세계' 개발에 리드
  - VR 기술을 활용하여 현실과 가상 세계 간의 융합을 추구하여 긍정적인 사용자 리뷰 획득
""",
        """
이력서 11

이름: 최승현
직군 및 직무: 게임 개발자
보유 스킬: Unreal Engine, C#, VR 게임 개발, AWS
경력사항:
- JKL 엔터테인먼트 (3년)
  - 가상 현실(VR) 기반의 어드벤처 게임 '환상의 세계' 개발에 리드
  - VR 기술을 활용하여 현실과 가상 세계 간의 융합을 추구하여 긍정적인 사용자 리뷰 획득
  - AWS 클라우드 배포
""",
        """
이력서 12

이름: 손재민
직군 및 직무: 가상현실(VR) 서버 개발자
보유 스킬: Node.js, Socket.io, MongoDB
경력사항:

RST 가상현실 기술 (3년)
VR 게임 및 애플리케이션을 위한 서버 아키텍처 개발 및 유지보수
다양한 VR 디바이스와의 호환성 확보를 위한 기술 연구
주요 성과:

'차원 간 탐험'이라는 VR RPG 게임에서의 서버 안정성 확보
사용자들 간의 협력 플레이 및 소셜 기능 도입으로 게임 커뮤니티 형성
""",
        """
이력서 13

이름: 김수현
직군 및 직무: 마케팅 전문가
보유 스킬: 디지털 마케팅, 소셜 미디어 관리, 콘텐츠 제작
경력사항:

ABC 디지털 에이전시 (5년)
다양한 고객을 대상으로 한 디지털 마케팅 전략 수립 및 실행
소셜 미디어 플랫폼을 활용한 캠페인 기획 및 진행
주요 성과:

클라이언트의 매출을 20% 증가시키는 데 성공한 캠페인 기획 및 실행
효과적인 콘텐츠 마케팅 전략 도입으로 브랜드 인지도 향상
""",
        """
이력서 14

이름: 이도훈
직군 및 직무: 가상현실(VR) 서버 개발자
보유 스킬: Java, Unity Networking, SQL
경력사항:

XYZ VR 테크놀로지 (4년)
VR 헤드셋 간의 실시간 데이터 통신을 위한 서버 아키텍처 설계 및 구현
사용자 경험을 개선하기 위한 네트워크 최적화 작업
주요 성과:

'가상 세계 탐험'이라는 VR MMORPG에서 다수의 플레이어 간의 원활한 상호작용 보장
효율적인 데이터베이스 관리로 게임 내 아이템 및 퀘스트 정보의 빠른 업데이트 가능
""",
        """
이력서 15

이름: 김지연
직군 및 직무: 인사 담당자
보유 스킬: 채용 프로세스 관리, 조직 문화 개선, 커뮤니케이션
경력사항:

HIJ 그룹 (6년)
전사적 채용 전략 수립 및 실시
조직 문화 개선을 위한 인사 정책 제안 및 실행
주요 성과:

채용 기간 단축 및 효율화를 통한 인력 채용 비용 절감
직원들 간의 워크샵 및 이벤트 개최로 조직 내 소통 강화
""",
        """
이력서 16

이름: 최미소
직군 및 직무: UI/UX 디자이너
보유 스킬: Adobe Creative Suite, Sketch, 웹 및 앱 디자인
경력사항:

KLM 미디어 디자인 (5년)
다양한 플랫폼을 위한 직관적이고 사용자 친화적인 인터페이스 디자인
고객과의 미팅 및 피드백을 통한 디자인 수정 및 개선
주요 성과:

새로운 미디어 플랫폼 런칭에 성공적으로 참여하여 사용자 증가
모바일 앱의 사용자 경험을 개선하여 리텐션률 15% 향상
""",
        """
이력서 17

이름: 한영희
직군 및 직무: 영상편집자
보유 스킬: Adobe Premiere Pro, After Effects, 색보정
경력사항:

QRS 미디어 프로덕션 (4년)
다양한 산업 분야의 비즈니스 및 광고 영상 편집
고객과의 미팅을 통한 프로젝트 요구사항 정의 및 수정
주요 성과:

광고 캠페인에 사용된 영상이 소비자들에게 긍정적인 반응
제품 프로모션 비디오 제작으로 기업의 매출 상승에 기여
""",
        """
이력서 18

이름: 김지은
직군 및 직무: VR 게임 개발자
보유 스킬: C++, Unreal Engine, VR 개발
경력사항:

XYZ VR 스튜디오 (4년)
고급 VR 시뮬레이션 게임 '차원의 신비'의 리드 프로그래머로 참여
VR 하드웨어 및 소프트웨어 간의 최적화 작업 진행
주요 성과:

게임 내 VR 요소를 활용하여 현실감 있는 경험 제공으로 사용자들의 호평 획득
VR 기술의 발전에 따라 팀 내에서 지속적인 습득 및 적용으로 최신 기술 도입에 기여
""",
        """
이력서 19

이름: 박준영
직군 및 직무: 클라우드 개발자
보유 스킬: AWS, Python, Docker, Kubernetes
경력사항:

ABC 클라우드 솔루션즈 (5년)
AWS 기반 클라우드 아키텍처 설계 및 구현
서버리스 아키텍처 도입으로 비용 효율성과 확장성 극대화
주요 성과:

기존 인프라를 클라우드로 이전하여 비용 절감 및 성능 향상 달성
새로운 서비스 론칭 시 클라우드 기술을 적극 활용하여 안정적인 운영 환경 구성
""",
        """
이력서 20

이름: 신현우
직군 및 직무: 클라우드 개발자
보유 스킬: Azure, Terraform, Jenkins, Ansible
경력사항:

XYZ 클라우드 인프라 (6년)
Azure 기반의 다양한 클라우드 서비스를 활용한 인프라 구축 및 관리
DevOps 프로세스 도입으로 지속적인 통합 및 배포 환경 구축
주요 성과:

기존 온프레미스 인프라를 클라우드로 전환하여 유지보수 비용 절감
팀 내에서 CI/CD 도구 도입 및 교육을 통한 개발 프로세스 향상
"""
]